In [41]:
import numpy as np
import pandas as pd
#pd.options.display.max_colwidth
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
import os
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import wfdb
import time
import random
from sklearn.preprocessing import minmax_scale
import sys
import torch

In [42]:
#pip install wfdb
#!pip install torch

In [43]:
##experiment with seed, run, channels(sinoatrial (SA) node, left feature lead, septal)
##later experiment with channel averaging of physician diagnosis features of myocardial infarction
##(septal average: v1, v2 ; left anterior average: v5, v6; left wrist: avl)
seed_num = 39 #sys.argv[1]  ##required bash files
#run_num = #sys.argv[2]
channel_1 = 'v6'#sys.argv[3]
channel_2 = 'vz'#sys.argv[4]
channel_3 = 'ii'#sys.argv[5]

In [44]:
with open('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/RECORDS') as fp:  
    lines = fp.readlines()

In [45]:
##lines   ##read lines in dataset records.txt

In [46]:
##divide into healthy and diseased .dat and .hea lists saved in files
files_myocardial = []
files_healthy = []

In [47]:
for file in lines:
    file_path = './ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1] + '.hea'
    
    ##read header to determine class ##focus on myocardial infarction
    if 'Myocardial infarction' in open(file_path).read():
        files_myocardial.append(file)
        
    if 'Healthy control' in open(file_path).read():
        files_healthy.append(file)

In [48]:
#files_healthy  ##check with CONTROL
#files_myocardial

In [49]:
##shuffle data (cross-validation)
np.random.seed(int(seed_num))
np.random.shuffle(files_myocardial) 
np.random.shuffle(files_healthy)

In [50]:
##split to train and test, 80% train, 20% test
healthy_train = files_healthy[:int(0.8*len(files_healthy))]
healthy_val = files_healthy[int(0.8*len(files_healthy)):]
myocardial_train = files_myocardial[:int(0.8*len(files_myocardial))]
myocardial_val = files_myocardial[int(0.8*len(files_myocardial)):]

In [51]:
##save names of train and val files
h_train = pd.DataFrame(healthy_train)
h_val = pd.DataFrame(healthy_val)
m_train = pd.DataFrame(myocardial_train)
m_val = pd.DataFrame(myocardial_val)

In [52]:
h_train.to_csv("h_train_idx.csv")
m_train.to_csv("m_train_idx.csv")
h_val.to_csv("h_val_idx.csv")
m_val.to_csv("m_val_idx.csv")

In [53]:
##remove intersections
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [54]:
patient_ids_healthy_train = []
patient_ids_healthy_val = []
patient_ids_myocardial_train = []
patient_ids_myocardial_val = []
##extract first 10 letters in file path string, which is the patient id
for index in healthy_train:
    patient_ids_healthy_train.append(index[0:10])
for index in healthy_val:
    patient_ids_healthy_val.append(index[0:10])
for index in myocardial_train:
    patient_ids_myocardial_train.append(index[0:10])
for index in myocardial_val:
    patient_ids_myocardial_val.append(index[0:10])

In [55]:
#type(patient_ids_myocardial_train)  ##list

In [56]:
len(np.unique(patient_ids_myocardial_train)) ##80% of total 148 patients is 118  
#likely to have duplicates, so distribute intersection over train and validation sets
#they can have common ecgs, used for checking

140

In [57]:
len(np.unique(patient_ids_myocardial_val)) ##20% of total 148 patients is 30

57

In [58]:
#patient_ids_myocardial_val ##stable

In [59]:
intersection_myocardial = intersection(patient_ids_myocardial_train, patient_ids_myocardial_val)
intersection_healthy = intersection(patient_ids_healthy_train, patient_ids_healthy_val)

In [60]:
##myocardial ##move half intersection to myocardial 
move_to_train = intersection_myocardial[:int(0.5*len(intersection_myocardial))]
move_to_val = intersection_myocardial[int(0.5*len(intersection_myocardial)):]

In [61]:
#move_to_train[0:5]  ##stable

In [62]:
for patient_id in move_to_train:
    in_val = []
    ##find and remove all files in val ecg readings by id
    for file_ in myocardial_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            myocardial_val.remove(file_)
            
    ##add to train
    for file_ in in_val:
        myocardial_train.append(file_)

for patient_id in move_to_val:    
    in_train = []
    ##find and remove all files in train
    for file_ in myocardial_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            myocardial_train.remove(file_)
            
    ##add to val
    for file_ in in_train:
        myocardial_val.append(file_)

In [63]:
##healthy
move_to_train = intersection_healthy[:int(0.5*len(intersection_healthy))]
move_to_val = intersection_healthy[int(0.5*len(intersection_healthy)):]
#print(move_to_train[0:5]) ##stable

In [64]:
for patient_id in move_to_train:
    in_val = []
    ##find and remove all files in val ecg readings by id
    for file_ in healthy_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            healthy_val.remove(file_)
            
    ##add to train
    for file_ in in_val:
        healthy_train.append(file_)

for patient_id in move_to_val:    
    in_train = []
    ##find and remove all files in train
    for file_ in healthy_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            healthy_train.remove(file_)
            
    ##add to val
    for file_ in in_train:
        healthy_val.append(file_)

In [65]:
len(np.unique(myocardial_val))

96

In [66]:
healthy_train[0][:-1]  ##example file directory

'patient240/s0468_re'

In [67]:
##update index files
h_train = pd.DataFrame(healthy_train)
h_val = pd.DataFrame(healthy_val)
m_train = pd.DataFrame(myocardial_train)
m_val = pd.DataFrame(myocardial_val)
h_train.to_csv("h_train_idx.csv")
m_train.to_csv("m_train_idx.csv")
h_val.to_csv("h_val_idx.csv")
m_val.to_csv("m_val_idx.csv")

In [68]:
##all files extracted and cleaned successfully
##now prepare a dataframe for the 3 channels
##Index----II----V6----VZ
data_healthy_train = []
for file in healthy_train:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_healthy_train.append(data)

In [69]:
data_healthy_train[0][:]

[array([-0.1245, -0.1275, -0.1305, ...,  0.188 ,  0.187 ,  0.1855]),
 array([ 0.458 ,  0.458 ,  0.457 , ..., -0.335 , -0.3325, -0.329 ]),
 array([-0.1025, -0.1   , -0.0925, ...,  0.062 ,  0.0695,  0.065 ])]

In [70]:
healthy_train_set = pd.DataFrame(data_healthy_train, columns=['ii', 'v6', 'vz'] )

In [71]:
healthy_train_set.at[0, 'ii'] ##input the first healthy patient in training

array([-0.1245, -0.1275, -0.1305, ...,  0.188 ,  0.187 ,  0.1855])

In [72]:
healthy_train_set.to_csv("healthy_train_signals.csv")

In [73]:
healthy_train_set.index  ##test for iterating over rows

RangeIndex(start=0, stop=61, step=1)

In [74]:
data_healthy_val = []
for file in healthy_val:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_healthy_val.append(data)

healthy_val_set = pd.DataFrame(data_healthy_train, columns=['ii', 'v6', 'vz'] )
healthy_val_set.to_csv("healthy_val_signals.csv")

In [75]:
data_unhealthy_train = []
for file in myocardial_train:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_unhealthy_train.append(data)

data_unhealthy_val = []
for file in myocardial_val:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_unhealthy_val.append(data)

unhealthy_train_set = pd.DataFrame(data_unhealthy_train, columns=['ii', 'v6', 'vz'] )
unhealthy_train_set.to_csv("mi_train_signals.csv")
unhealthy_val_set = pd.DataFrame(data_unhealthy_val, columns=['ii', 'v6', 'vz'] )
unhealthy_val_set.to_csv("mi_val_signals.csv")

In [76]:
##########DEBUGGING: TEST GENERATION OF SLICES
#unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_train))), k= 5)
#print(unhealthy_indices)
#unhealthy_batch = []
#for i in unhealthy_indices:
#    unhealthy_batch.append(data_unhealthy_train[i])

#print(unhealthy_batch[0][0])
#print(unhealthy_batch[0][1])
#print(unhealthy_batch[0][2])
##########DEBUGGING: TEST GENERATION OF SLICES

In [77]:
##############COMPONENT ONE IN .PY FILES##############
##normalize into batches, batch x and batch y (independent and dependent)
##minmax norm
window_size = 10000  ##from each ecg reading
def get_batch(batch_size, split='train'):
    ##random sampling of batches
    ##divide batch number in half to improve speed of network
    if split == 'train':
        ##random samples from sorted per batch with size k = batch_size / 2
        ##mimic shuffling
        unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_train))), k=int(batch_size / 2))
        healthy_indices = random.sample(sorted(np.arange(len(data_healthy_train))), k=int(batch_size / 2))
        unhealthy_batch = []
        healthy_batch = []
        for i in unhealthy_indices:
            unhealthy_batch.append(data_unhealthy_train[i])
        for j in healthy_indices:
            healthy_batch.append(data_healthy_train[j])
    elif split == 'val': 
        unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_val))), k=int(batch_size / 2))
        healthy_indices = random.sample(sorted(np.arange(len(data_healthy_val))), k=int(batch_size / 2))
        unhealthy_batch = []
        healthy_batch = []
        for i in unhealthy_indices:
            unhealthy_batch.append(data_unhealthy_val[i])
        for j in healthy_indices:
            healthy_batch.append(data_healthy_val[j])

    
    batch_x = []  ##batch of mixed healthy and unhealthy data
    for sample in unhealthy_batch: ##if val or if train
        start = random.choice(np.arange(len(sample[0]) - window_size))  ##randomly sample window from ecg 
        # normalize ecg values 
        normalized_1 = minmax_scale(sample[0][start:start+window_size])
        normalized_2 = minmax_scale(sample[1][start:start+window_size])
        normalized_3 = minmax_scale(sample[2][start:start+window_size])
        normalized = np.array((normalized_1, normalized_2, normalized_3))
        batch_x.append(normalized)
        
    for sample in healthy_batch:
        start = random.choice(np.arange(len(sample[0]) - window_size))
        # normalize
        normalized_1 = minmax_scale(sample[0][start:start+window_size])
        normalized_2 = minmax_scale(sample[1][start:start+window_size])
        normalized_3 = minmax_scale(sample[2][start:start+window_size])
        normalized = np.array((normalized_1, normalized_2, normalized_3))
        batch_x.append(normalized)
    
    batch_y = [0.1 for _ in range(int(batch_size / 2))]
    for _ in range(int(batch_size / 2)):
        batch_y.append(0.9)
        
    indices = np.arange(len(batch_y))
    np.random.shuffle(indices)
    
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    
    batch_x = batch_x[indices]
    batch_y = batch_y[indices]
    
    batch_x = np.reshape(batch_x, (-1, 3, window_size))
    batch_x = torch.from_numpy(batch_x)
    batch_x = batch_x.float()#.cuda()
    batch_x = batch_x.float()
    
    batch_y = np.reshape(batch_y, (-1, 1))
    batch_y = torch.from_numpy(batch_y)  ##save tensors
    batch_y = batch_y.float()#.cuda()
    batch_y = batch_y.float()
    
    return batch_x, batch_y

In [78]:
##constant batch size = 16  ##due to images ##or can be 10 
##test given train
##batch_x is the normalized windows of ecg channels in both healthy and unhealthy
batch_size = 10
batch_x, batch_y = get_batch(batch_size, split='train') 

In [79]:
##save batch tensors in .txt for ready training and testing
##assume 80 training batches and 20 testing batches ##each batch has a size of 5 or 8
torch.save(batch_x, 'batch_x.pt')  ##encode in utf_8
torch.save(batch_y, 'batch_y.pt')  ##encode in utf_8

In [80]:
##in model, import the get_batches from this module then start
torch.load('batch_x.pt')

tensor([[[0.1437, 0.1421, 0.1405,  ..., 0.2667, 0.2667, 0.2663],
         [0.5625, 0.5632, 0.5619,  ..., 0.4991, 0.4997, 0.4991],
         [0.0913, 0.0899, 0.0902,  ..., 0.3420, 0.3412, 0.3471]],

        [[0.0657, 0.0600, 0.0600,  ..., 0.3689, 0.3668, 0.3673],
         [0.3036, 0.2942, 0.2834,  ..., 0.2790, 0.2790, 0.2794],
         [0.2583, 0.2592, 0.2610,  ..., 0.2350, 0.2278, 0.2305]],

        [[0.2842, 0.2836, 0.2896,  ..., 0.2085, 0.2112, 0.2118],
         [0.2325, 0.2298, 0.2312,  ..., 0.1868, 0.1734, 0.1694],
         [0.2322, 0.2487, 0.2690,  ..., 0.3938, 0.4040, 0.4035]],

        ...,

        [[0.3166, 0.3161, 0.3161,  ..., 0.1739, 0.1723, 0.1675],
         [0.1681, 0.1643, 0.1634,  ..., 0.1709, 0.1709, 0.1709],
         [0.3528, 0.3549, 0.3555,  ..., 0.1548, 0.1575, 0.1585]],

        [[0.0142, 0.0154, 0.0156,  ..., 0.6785, 0.6775, 0.6779],
         [0.4780, 0.4752, 0.4757,  ..., 0.3006, 0.2987, 0.2992],
         [0.7798, 0.7808, 0.7801,  ..., 0.0454, 0.0466, 0.0449]],

 

In [81]:
torch.load('batch_y.pt')

tensor([[0.9000],
        [0.9000],
        [0.1000],
        [0.9000],
        [0.1000],
        [0.9000],
        [0.1000],
        [0.1000],
        [0.1000],
        [0.9000]])

In [83]:
import sys
import numpy  ##disable truncations
#numpy.set_printoptions(threshold=sys.maxsize)

In [90]:
data_healthy_train[0][0][0:53]

array([-0.1245, -0.1275, -0.1305, -0.134 , -0.134 , -0.1345, -0.1345,
       -0.1345, -0.132 , -0.1295, -0.128 , -0.1225, -0.121 , -0.123 ,
       -0.128 , -0.132 , -0.13  , -0.127 , -0.13  , -0.133 , -0.1305,
       -0.1235, -0.1205, -0.121 , -0.121 , -0.1205, -0.1205, -0.122 ,
       -0.123 , -0.1215, -0.1165, -0.1125, -0.1145, -0.117 , -0.1175,
       -0.115 , -0.105 , -0.1   , -0.1095, -0.114 , -0.114 , -0.1145,
       -0.1115, -0.104 , -0.103 , -0.1075, -0.1105, -0.114 , -0.1135,
       -0.1115, -0.108 , -0.107 , -0.114 ])

In [89]:
data_healthy_train[0][1][0:53]

array([0.458 , 0.458 , 0.457 , 0.4555, 0.453 , 0.45  , 0.449 , 0.45  ,
       0.448 , 0.4455, 0.4465, 0.4505, 0.4515, 0.4535, 0.4585, 0.4605,
       0.4655, 0.4715, 0.4715, 0.4715, 0.4765, 0.4795, 0.4795, 0.482 ,
       0.487 , 0.4875, 0.485 , 0.483 , 0.4805, 0.479 , 0.4795, 0.482 ,
       0.483 , 0.4795, 0.4785, 0.476 , 0.472 , 0.473 , 0.4785, 0.484 ,
       0.4825, 0.4785, 0.4725, 0.4625, 0.459 , 0.457 , 0.4535, 0.4565,
       0.4605, 0.465 , 0.4725, 0.477 , 0.481 ])

In [91]:
data_healthy_train[0][2][0:53]

array([-0.1025, -0.1   , -0.0925, -0.0885, -0.0905, -0.0905, -0.0895,
       -0.085 , -0.08  , -0.0825, -0.0835, -0.067 , -0.064 , -0.0665,
       -0.064 , -0.0685, -0.0695, -0.082 , -0.0865, -0.085 , -0.089 ,
       -0.0855, -0.087 , -0.091 , -0.091 , -0.092 , -0.09  , -0.0805,
       -0.0745, -0.075 , -0.0705, -0.07  , -0.081 , -0.086 , -0.0935,
       -0.105 , -0.0935, -0.08  , -0.0865, -0.0795, -0.073 , -0.0835,
       -0.0895, -0.0915, -0.0885, -0.0925, -0.1015, -0.105 , -0.105 ,
       -0.1005, -0.1035, -0.109 , -0.113 ])